#**Despliegue**

Realizado por: Iván Tang Zhu

Datos: futuro

- Cargamos el modelo
- Cargamos los datos futuros
- Preparar los datos futuros
- Aplicamos el modelo para la predicción

- Pasos
  - No se pueden colocar celdas de texto
  - En este notebook, se ejecuta un despliegue funcional
  - ***No se ejecuta streamlit aqui***
  - Se va streamlit https://streamlit.io/cloud
  - Login con github
  - Se indica repositorio que debe de tener el app.py (este notebook), requirements (para que instale las librerías utilizadas) y el archivo .pkl con los modelos guardados
  - Esto crea un enlace *temporal* con la interfaz gráfica del despliegue. Esto se puede compartir a otros

In [ ]:
#Importamos librerías básicas
import pandas as pd # manipulacion dataframes
import numpy as np  # matrices y vectores
import matplotlib.pyplot as plt #gráfica

In [ ]:
#Paso 1: Limpiar atípicos
def remove_outliers(X):
    #Se asigna valor nulo a los atípicos

    X.loc[(X["u"] < 9) | (X["u"] >30), "u"] = np.nan
    X.loc[(X["g"] < 9) | (X["g"] >30), "g"] = np.nan
    X.loc[(X["r"] < 9) | (X["r"] >30), "r"] = np.nan
    X.loc[(X["i"] < 9) | (X["i"] >30), "i"] = np.nan
    X.loc[(X["z"] < 9) | (X["z"] >30), "z"] = np.nan


    #Validar todas las variables

    return X

In [ ]:
#Cargamos el modelo
import pickle
filename = 'pipeline_modelo.pkl'
pipeline_model, labelencoder = pickle.load(open(filename, 'rb'))
#pipeline_model

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('outliers',
                                  FunctionTransformer(func=<function remove_outliers at 0x78f9a6ea2e80>)),
                                 ('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer())]),
                                                                   ['u', 'g',
                                                                    'r', 'i',
                                                                    'z',
                                                                    'redshift'])]))])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=63,
                                        min_samples_leaf=19))])

In [ ]:
#Cargamos los datos futuros
# data = pd.read_csv("datos_futuro.csv")
# data.head()

,u,g,r,i,z,redshift
0,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794
1,20.11234,19.76321,19.32145,19.14200,19.03542,0.000012
2,21.54320,20.85432,20.11289,19.73102,19.50487,0.423568
3,22.78456,21.98765,21.10342,20.45231,20.23107,1.832451
4,19.45328,19.14273,18.93742,18.81231,18.76450,1.434147


In [ ]:
#Se crea interfaz gráfica con streamlit para captura de los datos
#Recordar: Esto no se ejecuta aqui
import streamlit as st

st.title('Predicción de Objetos Estelares')

u = st.slider('Filtro ultravioleta (u)', min_value=0.0, max_value=30.0, value=20.0, step=0.00001)
g = st.slider('Filtro verde (g)', min_value=0.0, max_value=30.0, value=20.0, step=0.00001)
r = st.slider('Filtro rojo (r)', min_value=0.0, max_value=30.0, value=20.0, step=0.00001)
i = st.slider('Filtro infrarrojo cercano (i)', min_value=0.0, max_value=30.0, value=20.0, step=0.00001)
z = st.slider('Filtro infrarrojo (z)', min_value=0.0, max_value=30.0, value=20.0, step=0.00001)
redshift = st.slider('Corregimiento al rojo (redshift)', min_value=-0.1, max_value=7.0, value=3.0, step=0.00001, format="%.5f")

#Dataframe
datos = [[u, g, r, i, z, redshift]]
data = pd.DataFrame(datos, columns=['u', 'g', 'r', 'i', 'z', 'redshift']) #Dataframe con los mismos nombres de variables

# **Predicciones**

- Utilizando un Árbol de decisión optimizado con F1 Score del 0,97

In [ ]:
#Hacemos la predicción con el Tree
Y_Tree = pipeline_model.predict(data)
print(Y_Tree)

[0 2 1 1 1]


In [ ]:
print(labelencoder.inverse_transform(Y_Tree)) # Devuelve a los nombre originales

['GALAXY' 'STAR' 'QSO' 'QSO' 'QSO']


In [ ]:
data['Predicción']=labelencoder.inverse_transform(Y_Tree)
data.head()

,u,g,r,i,z,redshift,Predicción
0,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY
1,20.11234,19.76321,19.32145,19.14200,19.03542,0.000012,STAR
2,21.54320,20.85432,20.11289,19.73102,19.50487,0.423568,QSO
3,22.78456,21.98765,21.10342,20.45231,20.23107,1.832451,QSO
4,19.45328,19.14273,18.93742,18.81231,18.76450,1.434147,QSO


In [ ]:
data

,u,g,r,i,z,redshift,Predicción
0,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY
1,20.11234,19.76321,19.32145,19.14200,19.03542,0.000012,STAR
2,21.54320,20.85432,20.11289,19.73102,19.50487,0.423568,QSO
3,22.78456,21.98765,21.10342,20.45231,20.23107,1.832451,QSO
4,19.45328,19.14273,18.93742,18.81231,18.76450,1.434147,QSO
